In [1]:
from os import environ as ENV
from time import time as now
import json

import sys
sys.path.append(ENV['PWD'])

In [2]:
from circuit_extra import *

qc, qc_qasm = test_qiskit_circuit()

In [3]:
from provider_ionq import IONQ
ionq = IONQ(
    ionq_api_key := open(ENV['HOME'] + "/work/api-keys/IONQ_API_KEY").readline().strip()
)

/home/user/work/quadrigems/.venv/lib/python3.11/site-packages/qbraid/_entrypoints.py:20: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [4]:
from provider_ibmq import IBMQ
ibmq = IBMQ(
    ibm_api_key := open(ENV['HOME'] + '/work/api-keys/IBM_API_KEY').readline().strip(),
    ibm_crn := open(ENV['HOME'] + '/work/api-keys/IBM_CRN').readline().strip()
)

In [16]:
ibmq.printqpus()
ionq.printqpus()

ibmq QiskitBackend('ibm_kingston') 156qb DeviceStatus.ONLINE
ibmq QiskitBackend('ibm_fez') 156qb DeviceStatus.ONLINE
ibmq QiskitBackend('ibm_marrakesh') 156qb DeviceStatus.ONLINE
ibmq QiskitBackend('ibm_torino') 133qb DeviceStatus.ONLINE
ibmq QiskitBackend('ibm_brisbane') 127qb DeviceStatus.ONLINE
ibmq QiskitBackend('ibm_sherbrooke') 127qb DeviceStatus.ONLINE
ionq IonQDevice('qpu.harmony') 11qb DeviceStatus.RETIRED now!
ionq IonQDevice('qpu.aria-1') 25qb DeviceStatus.ONLINE queue 298d7h36m11s
ionq IonQDevice('qpu.aria-2') 25qb DeviceStatus.UNAVAILABLE queue 2288d10h2m14s
ionq IonQDevice('qpu.forte-1') 36qb DeviceStatus.UNAVAILABLE queue 65d2h8m35s
ionq IonQDevice('qpu.forte-enterprise-1') 36qb DeviceStatus.ONLINE queue 587d15h10m36s
ionq IonQDevice('qpu.forte-enterprise-2') 36qb DeviceStatus.UNAVAILABLE now!
ionq IonQDevice('simulator') 29qb DeviceStatus.ONLINE now!


# IonQ

In [5]:
# cache recently run jobs
try:
    jobs_ionq
    with open(ENV['HOME'] + '/work/jobs_ionq.json', 'wt+') as f:
        json.dump({
            ibmq.jobid(v): k for k, v in jobs_ionq.items()
        }, fp=f, indent=4)
except:
    with open(ENV['HOME'] + '/work/jobs_ionq.json', 'rt') as f:
        jobs_ionq = {
            time: job_id
            for job_id, time in json.load(fp=f).items()
        }

jobs_ionq

{0.0: '0197faea-587a-769d-ac6c-c916cbad46a4',
 1752294901.9915304: '0197fca5-fd59-77ca-8a93-552f7f197978',
 1752320373.48551: '0197fe6e-f53e-7214-a11f-ebd495daf9f6',
 1752321436.848685: '0197fe7f-2f3d-75a3-9c91-f84a31b6c478'}

In [6]:
# get qpu
ionq_qpu = ionq.provider.get_device('simulator')#('qpu.forte-enterprise-1')
ionq_qpu.status()

<DeviceStatus.ONLINE: 'online'>

In [7]:
if use_cached_job := True:
    t = 1752321436.848685
    jid = jobs_ionq[t]
else:
    t = now()
    jj = ionq_qpu.run(qc, name='test', shots=20) #, preflight=True)
    jid = jobs_ionq[t] = jj.id

j = ionq.job(jid)
r = ionq.jobresult(jid, raw=True)

In [8]:
print(f"Job: {jid}")
try: jj.wait_for_final_state()
except: pass
print(f"Cost Estimate (USD): {j['cost_usd']}")
print(f"Result: {int(ionq.job(jid).get('shots', 0))}x")
print(json.dumps(r, indent=4))

Job: 0197fe7f-2f3d-75a3-9c91-f84a31b6c478
Cost Estimate (USD): 0
Result: 0x
{
    "000": 0.5,
    "111": 0.5
}


In [9]:
print(ionq.jobresult(jid))

{'000': 0.5, '111': 0.5}


In [10]:
ionq.jobcircuit(jid).draw(fold=-1)

┌───┐           ░ ┌─┐      
q2_0: ┤ H ├──■────■───░─┤M├──────
      └───┘┌─┴─┐  │   ░ └╥┘┌─┐   
q2_1: ─────┤ X ├──┼───░──╫─┤M├───
           └───┘┌─┴─┐ ░  ║ └╥┘┌─┐
q2_2: ──────────┤ X ├─░──╫──╫─┤M├
                └───┘ ░  ║  ║ └╥┘
c1: 3/═══════════════════╩══╩══╩═
                         0  1  2

# IBM

In [11]:
# cache recently run jobs
try:
    jobs_ibmq
    with open(ENV['HOME'] + '/work/jobs_ibmq.json', 'wt+') as f:
        json.dump({
            ibmq.jobid(v): k for k, v in jobs_ibmq.items()
        }, fp=f, indent=4)
except:
    with open(ENV['HOME'] + '/work/jobs_ibmq.json', 'rt') as f:
        jobs_ibmq = {
            time: job_id
            for job_id, time in json.load(fp=f).items()
        }

jobs_ibmq

{1752280358.6594968: 'd1oqoogt0npc73fm0dt0',
 1752280416.8679636: 'd1oqqob3rr0s73bgjbdg'}

In [12]:
# get qpu
ibmq_qpu = ibmq.provider.get_device('ibm_kingston')
ibmq_qpu.status()

<DeviceStatus.ONLINE: 'online'>

In [13]:
if use_cached_job := True:
    t = 1752280416.8679636
    jid = jobs_ibmq[t]
else:
    t = now()
    jj = ibmq_qpu.run(qasm, shots=20)
    jid = jobs_ibmq[t] = jj.id

j = ibmq.job(jid)
r = ibmq.jobresult(jid, raw=False)

In [14]:
print(f"Job: {jid}")
try: jj.wait_for_final_state()
except: pass
print(f"Cost: {j.metrics()['bss']['seconds']}s")
print(f"Result: {j.result()._metadata['execution']['execution_spans'][0].size}x")
print(json.dumps(r, indent=4))

Job: d1oqqob3rr0s73bgjbdg
Cost: 1s
Result: 20x
{
    "000": 7,
    "111": 11,
    "010": 1,
    "001": 1
}


In [15]:
ibmq.jobcircuit(jid).trim().draw(fold=-1)

┌──────────┐┌────┐ ┌────────────┐┌────┐┌─────────┐┌──────┐ ┌──────────┐ ┌────┐┌─────────────┐┌──────┐┌───┐┌─┐
 q41: ┤ Rz(-π/2) ├┤ √X ├─┤ Rz(-1.296) ├┤ √X ├┤ Rz(π/2) ├┤1     ├─┤ Rz(-π/2) ├─┤ √X ├┤ Rz(-2.8668) ├┤0     ├┤ X ├┤M├
      └┬────────┬┘├────┤ └─┬────────┬─┘└────┘└─────────┘│      │ └──────────┘ └────┘└─────────────┘│  Ecr │└┬─┬┘└╥┘
 q53: ─┤ Rz(-π) ├─┤ √X ├───┤ Rz(-π) ├───────────────────┤  Ecr ├───────────────────────────────────┤1     ├─┤M├──╫─
      ┌┴────────┤ ├────┤┌──┴────────┴─┐                 │      │┌────────────┐┌────┐  ┌──────────┐ └─┬─┬──┘ └╥┘  ║ 
 q42: ┤ Rz(π/2) ├─┤ √X ├┤ Rz(-1.5302) ├─────────────────┤0     ├┤ Rz(-3.101) ├┤ √X ├──┤ Rz(-π/2) ├───┤M├─────╫───╫─
      └─────────┘ └────┘└─────────────┘                 └──────┘└────────────┘└────┘  └──────────┘   └╥┘     ║   ║ 
c0: 1/════════════════════════════════════════════════════════════════════════════════════════════════╬══════╬═══╩═
                                                                                                      ║      ║   0 
c1: 1/════════════════════════════════════════════════════════════════════════════════════════════════╩══════╬═════
                                                                                                      0      ║     
c2: 1/═══════════════════════════════════════════════════════════════════════════════════════════════════════╩═════
                                                                                                             0